In [2]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

def augment_data(src_dir, dest_dir, num_augmented_images):
    datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    image_filenames = os.listdir(src_dir)
    total_images = len(image_filenames)

    if total_images == 0:
        print(f"No images found in {src_dir}.")
        return

    images_to_generate_per_image = num_augmented_images // total_images

    for image_filename in image_filenames:
        img_path = os.path.join(src_dir, image_filename)
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=dest_dir, save_prefix='aug', save_format='jpeg'):
            i += 1
            if i >= images_to_generate_per_image:
                break

# Paths to the original and augmented directories
litter_src_dir = 'dog dataset/dog dataset/Litter'
no_litter_src_dir = 'dog dataset/dog dataset/No litter'
litter_aug_dir = 'dog dataset/dog dataset/Litter_augmented'
no_litter_aug_dir = 'dog dataset/dog dataset/No litter_augmented'

# Number of augmented images to generate
num_augmented_images = 1000  # Adjust as needed

# Augment the minority class
augment_data(litter_src_dir, litter_aug_dir, num_augmented_images)
augment_data(no_litter_src_dir, no_litter_aug_dir, num_augmented_images)

# Define the path to your combined dataset
combined_data_path = 'dog dataset_combined'

# Create directories for combined dataset
os.makedirs(os.path.join(combined_data_path, 'Litter'), exist_ok=True)
os.makedirs(os.path.join(combined_data_path, 'No litter'), exist_ok=True)

# Copy original and augmented images to combined dataset directories
for src_dir, dest_dir in [(litter_src_dir, os.path.join(combined_data_path, 'Litter')),
                          (no_litter_src_dir, os.path.join(combined_data_path, 'No litter')),
                          (litter_aug_dir, os.path.join(combined_data_path, 'Litter')),
                          (no_litter_aug_dir, os.path.join(combined_data_path, 'No litter'))]:
    for filename in os.listdir(src_dir):
        src_path = os.path.join(src_dir, filename)
        dest_path = os.path.join(dest_dir, filename)
        if not os.path.exists(dest_path):  # Avoid overwriting
            os.link(src_path, dest_path)

# Create ImageDataGenerator for training and validation with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # 20% for validation

# Training data generator
train_generator = train_datagen.flow_from_directory(
    combined_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')  # Use subset for training

# Validation data generator
validation_generator = train_datagen.flow_from_directory(
    combined_data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')  # Use subset for validation

# Load MobileNetV2 with pre-trained ImageNet weights, excluding the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Combine the base model with the new layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers to retain pre-trained features
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


Found 6844 images belonging to 2 classes.
Found 1710 images belonging to 2 classes.
Epoch 1/10


c:\Users\shant\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


213/213 ━━━━━━━━━━━━━━━━━━━━ 269s 1s/step - accuracy: 0.9033 - loss: 0.2658 - val_accuracy: 0.9965 - val_loss: 0.0136
Epoch 2/10
  1/213 ━━━━━━━━━━━━━━━━━━━━ 1:02 297ms/step - accuracy: 0.9688 - loss: 0.2006

c:\Users\shant\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9688 - loss: 0.2006 - val_accuracy: 1.0000 - val_loss: 0.0266
Epoch 3/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 154s 712ms/step - accuracy: 0.9898 - loss: 0.0291 - val_accuracy: 0.9994 - val_loss: 0.0040
Epoch 4/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 909us/step - accuracy: 0.9688 - loss: 0.0998 - val_accuracy: 1.0000 - val_loss: 5.6344e-05
Epoch 5/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 153s 705ms/step - accuracy: 0.9948 - loss: 0.0156 - val_accuracy: 0.9900 - val_loss: 0.0250
Epoch 6/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 1s 946us/step - accuracy: 0.9688 - loss: 0.1156 - val_accuracy: 1.0000 - val_loss: 0.0098
Epoch 7/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 188s 869ms/step - accuracy: 0.9910 - loss: 0.0247 - val_accuracy: 0.9994 - val_loss: 0.0034
Epoch 8/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 2.4516e-04
Epoch 9/10
213/213 ━━━━━━━━━━━━━━━━━━━━ 155s 717ms/step - accuracy: 0.9961 - loss: 0.0141

In [9]:

# Path to the test dataset
test_path = 'test dataset'

# Function to preprocess a single image
def preprocess_image(img_path):
    img = load_img(img_path, target_size=(224, 224))  # Load image with target size
    img_array = img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to fit model input
    img_array /= 255.0  # Normalize the image
    return img_array

# Get the first 10 images from the test dataset
test_images = [os.path.join(test_path, fname) for fname in os.listdir(test_path)]

# Make predictions on the first 10 test images
for img_path in test_images:
    img = preprocess_image(img_path)
    prediction = model.predict(img)
    class_label = 'Litter' if prediction < 0.5 else 'No litter'
    print(f"Image: {img_path} - Predicted: {class_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Image: test dataset\-1-_jpg.rf.03067c74d1c62463feda5c096800dd9d.jpg - Predicted: No litter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Image: test dataset\-1-_jpg.rf.3bb5d6bf63b24427d2433e5fa2dd6c9e.jpg - Predicted: No litter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Image: test dataset\-1-_jpg.rf.a198c9e6a3f34e862d912469bf64d7d6.jpg - Predicted: No litter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Image: test dataset\-1-_jpg.rf.a956975234c9b1ce0daa5f6203036ae1.jpg - Predicted: No litter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Image: test dataset\-1-_jpg.rf.b0be7e51c2a2166a43daa8b4f259f16e.jpg - Predicted: No litter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Image: test dataset\-1-_jpg.rf.cfaa4a9f8d5b44a8e3341d869ee44571.jpg - Predicted: No litter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Image: test dataset\-1-_jpg.rf.d932431fb6ae3e4e915895bca352c7c9.jpg - Predicted: No litter
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Image: test dataset\-1-_jpg.rf.db7347ab8e77b463bcad0725252d

In [10]:
model.save('model1.h5')